In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

/opt/anaconda3/envs/torch8019/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#loss function參考1
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Load VGG16 model as the backbone
        self.backbone = models.vgg16(pretrained=True)
        # Remove the last layer (the classification layer) of VGG16
        self.backbone.classifier = nn.Sequential(*list(self.backbone.classifier.children())[:-1])
        # Add a fully connected layer with 512 output units for Siamese network
        self.fc = nn.Linear(512, 1)

    def forward_once(self, x):
        # Pass input image through the VGG16 backbone
        output = self.backbone(x)
        # Flatten the output
        output = output.view(output.size()[0], -1)
        # Pass through the fully connected layer
        output = self.fc(output)
        return output

    def forward(self, x1, x2):
        # Pass the first image through the network
        output1 = self.forward_once(x1)
        # Pass the second image through the network
        output2 = self.forward_once(x2)
        # Return the absolute difference between the two outputs
        distance = torch.abs(output1 - output2)
        return distance

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, distance, label):
        # Compute the contrastive loss
        loss = torch.mean((1 - label) * torch.pow(distance, 2) +
                          (label) * torch.pow(torch.clamp(self.margin - distance, min=0.0), 2))
        return loss

In [ ]:
#loss function參考2
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, target):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - target) * torch.pow(euclidean_distance, 2) +
                                      target * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [ ]:
#loss function參考3
class ContrastiveLoss(nn.Module):
    """
    Contrastive loss
    Takes embeddings of two samples and a target label == 1 if samples are from the same class and label == 0 otherwise
    """

    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, output1, output2, target, size_average=True):
        distances = (output2 - output1).pow(2).sum(1)  # squared distances
        losses = 0.5 * (target.float() * distances +
                        (1 + -1 * target).float() * F.relu(self.margin - (distances + self.eps).sqrt()).pow(2))
        return losses.mean() if size_average else losses.sum()

In [11]:
#產生圖片的Embedding
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.features = models.vgg16(pretrained=True)
        # ------------------------------------------------------------------------------
        # Remove the last layer (the classification layer) of VGG16 第一種寫法 把原本VGG的最後一層fc layer刪掉 直接線性輸出
        # self.backbone.classifier = nn.Sequential(*list(self.backbone.classifier.children())[:-1])
        # self.features = self.backbone.classifier
        # self.fc = nn.Linear(512, 1)
        # ------------------------------------------------------------------------------
        # 第二種寫法 
        self.features.classifier = nn.Sequential()
        self.fc = nn.Sequential(
            nn.Linear(512*7*7, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 128),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(128, 10),#原num_classes 這裡要用多少維輸出來代表這個item
        )
    def forward(self, x):
        #經過VGG提取特徵
        output = self.backbone(x)
        #flatten
        output = output.view(output.size()[0], -1)
        #經過fc層
        output = self.fc(x)
        return output
        
    def get_embedding(self, x):
        return self.forward(x)


In [12]:
print(EmbeddingNet())    

/opt/anaconda3/envs/torch8019/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/anaconda3/envs/torch8019/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EmbeddingNet(
  (features): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
    

In [ ]:
class SiameseNet(nn.Module):
    def __init__(self, EmbeddingNet):
        super(SiameseNet, self).__init__()
        self.EmbeddingNet = EmbeddingNet
        
    def forward(self, x1, x2):
        output1 = self.EmbeddingNet(x1)
        output2 = self.EmbeddingNet(x2)
        return output1, output2

    # def get_embedding(self, x):
    #     return self.EmbeddingNet(x)

In [13]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2): #margin要自己定義 2
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        #self.eps = 1e-9
    
    #這邊可以改成不同距離計算方式 這裡先用歐幾里德距離計算
    def forward(self, output1, output2, target):
        distance = F.pairwise_distance(output1, output2, p=2)
        loss = torch.mean((1 - target) * torch.pow(distance, 2) + target * torch.pow(torch.clamp(self.margin - distance, min=0.0), 2))
        return loss

IndentationError: expected an indented block (3052328748.py, line 3)